In [33]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
data = pd.read_csv('Tweets.csv')

In [35]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [36]:
data = data[['airline_sentiment', 'text']]

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  object
 1   text               14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [38]:
data.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [39]:
data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [40]:
token = re.compile(r'[A-Za-z]+|[!?.:,()]')

In [41]:
def extract_text(text):
    new_text = token.findall(text)
    new_text = ' '.join([x.lower() for x in new_text])
    return new_text

In [42]:
x = data.text.apply(extract_text)

In [43]:
y = data.airline_sentiment

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [45]:
vect = TfidfVectorizer(ngram_range=(1, 3), stop_words='english', min_df=3)

In [46]:
x_train_vect = vect.fit_transform(x_train)

In [47]:
 
x_test_vect = vect.transform(x_test)


In [48]:
model = RandomForestClassifier()

In [49]:
model.fit(x_train_vect, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [50]:
model.score(x_train_vect, y_train)

0.9939890710382514

In [51]:
# 过拟合
model.score(x_test_vect, y_test)

0.7530054644808744

In [52]:
model2 = RandomForestClassifier(n_estimators=500)

In [53]:
model2.fit(x_train_vect, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [54]:
model2.score(x_train_vect, y_train)

0.9939890710382514

In [55]:
model2.score(x_test_vect, y_test)

0.7527322404371585

In [56]:
param = {
    'max_depth': range(1, 500, 10),
    'criterion': ['gini', 'entropy']
}

In [58]:
grid_s = GridSearchCV(estimator=RandomForestClassifier(n_jobs=8),
                      param_grid=param,
                      cv=5)

In [59]:
x_vect = vect.transform(x)

In [ ]:
grid_s.fit(x_vect, y)